# Analyse av datasett
Notebooken bruker funksjoner fra klassene i analyseData-mappen og viser eksempler
på bruksområdet av de forskjellige funksjonene

## Felles import for alle kodeblokker

In [1]:
import sys

# Sørger for at vi alltid importerer fra ../analyseData,
# i tilfelle denne cellen kjøres flere ganger ettersom
# at arbeidsmappen er endret til ../missingData senere.
sys.path.append("../../src/analyseData")

from yearlystats import YearlyStats
from outlieranalysis import OutlierAnalysis
from monthlystats import MonthlyStats

ms = MonthlyStats("../../data/processed")
ys = YearlyStats("../../data/processed")
oa = OutlierAnalysis("../../data/processed")

# Import er delt i to da de henter fra forskjellige mapper
sys.path.append("../../src/missingData")
from missingdatafinder import MissingDataConverter
missing = MissingDataConverter()

## Eksempel koder på bruk av analyse funksjonene

### Viser statistikk ved en eller alle måneder

In [2]:
# Regn ut statistikk for én måned i Oslo
result = ms.compute_single_month(
    "2024-03", "mean(air_temperature P1D)", "oslo"
)
print(result)

# Hent alle måneder for Tromsø
df = ms.compute_all_months(
    "range(air_temperature P1D)", "tromso"
)
df.head()

{'mean': 2.6967741935483875, 'median': 2.7, 'std': 2.2953647360696494}


,year_month,mean,median,std
0,2000-01,4.210484,3.8250,1.404602
1,2000-02,4.473276,4.3500,1.378857
2,2000-03,4.380645,3.8750,1.605131
3,2000-04,5.159167,4.9375,1.689961
4,2000-05,5.586290,5.5250,1.646965


### Finner antall ekstremverdier

In [3]:
df_outliers = oa.find_outliers_per_month(
    city="oslo",
    element_id="sum(precipitation_amount P1D)",
    include_empty_months=True
)
df_outliers.head()

,year_month,outliers_removed,total_count,outlier_percentage
0,2000-01,4,31,12.9
1,2000-02,3,29,10.3
2,2000-03,3,31,9.7
3,2000-04,1,30,3.3
4,2000-05,0,31,0.0


### Finner statistiske verdier med og uten ekstremverdier

In [4]:
df_stats = oa.stats_with_without_outliers(
    city="tromso",
    element_id="mean(air_temperature P1D)",
    statistic="mean"
)
df_stats.head()

,year_month,mean_with_outliers,mean_without_outliers,outliers_removed,element_id
0,2000-01,-2.400,-2.400,0,mean(air_temperature P1D)
1,2000-02,-3.297,-3.297,0,mean(air_temperature P1D)
2,2000-03,-3.113,-3.113,0,mean(air_temperature P1D)
3,2000-04,-0.810,-0.810,0,mean(air_temperature P1D)
4,2000-05,4.965,4.965,0,mean(air_temperature P1D)


### Henter og viser manglende verdier i en DataFrame

In [5]:
df_missing = missing.read_missing_values("../../data/missing/missing_in_both.csv")
df_missing.head()

,date,timeOffset,elementId,missing
0,2003-06-01,PT0H,max(air_temperature P1D),Oslo
1,2003-06-01,PT0H,min(air_temperature P1D),Oslo
2,2003-06-01,PT0H,range(air_temperature P1D),Oslo
3,2003-06-02,PT0H,max(air_temperature P1D),Oslo
4,2003-06-02,PT0H,min(air_temperature P1D),Oslo


### Viser prosentvis endring fra en periode til neste

In [6]:
df_change = ys.percent_change(
    city="oslo",
    element_id="sum(precipitation_amount P1D)",
    time_offset="PT18H",
    statistic="mean",
    frequency="YE",  # "D" for dag, "ME" for måned, "YE" for år
    start="2020-01",
    end="2024-01"
)
df_change.head()


,period,value,percent_change
0,2021-12-31 00:00:00+00:00,1.908219,-35.302620
1,2022-12-31 00:00:00+00:00,1.922466,0.746590
2,2023-12-31 00:00:00+00:00,2.802740,45.788799
